In [ ]:
pip --version

pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=19fe6f84a998bc48235b16a544c9ca18bbd580253560a8217cf06c16e91ad7af
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [24]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.1 MB/s eta 0:00:00


In [25]:
import csv
from faker import Faker
import random

fake = Faker()

num_records = 100000

http_methods = ['GET', 'POST', 'PUT', 'DELETE']
response_codes = [200, 301, 404, 500]

file_path = "web_server_logs.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ip', 'timestamp', 'method', 'url', 'response_code', 'response_size'])

    for _ in range(num_records):
        ip = fake.ipv4()
        timestamp = fake.date_time_this_year().isoformat()
        method = random.choice(http_methods)
        url = fake.uri_path()
        response_code = random.choice(response_codes)
        response_size = random.randint(100, 10000)

        writer.writerow([ip, timestamp, method, url, response_code, response_size])

print(f"Сгенерировано {num_records} записей и сохранено в {file_path}")

Сгенерировано 100000 записей и сохранено в web_server_logs.csv


In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# Создание SparkSession
spark = SparkSession.builder.appName("Final task").getOrCreate()

# Чтение CSV-файла
logs_df = spark.read.csv("web_server_logs.csv", header=True, inferSchema=True)

# Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP
top_ip = logs_df.groupBy("ip").count().orderBy(col("count").desc()).limit(10)
print('Топ 10 активных IP адресов:')
top_ip.show()

# Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.
count_response_for_method = logs_df.groupBy("method").count().orderBy(col("count").desc())
print("Количество запросов для каждого метода:")
count_response_for_method.show()

# Профильтруйте и посчитайте количество запросов с кодом ответа 404.
count_response_404 = logs_df.filter(col("response_code") == 404).count()
print(f"Количество запросов с кодом ответа 404: {count_response_404}")

# Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.
response_size_per_date = logs_df.withColumn("timestamp", to_date("timestamp", "yyyy-MM-dd"))\
.withColumnRenamed("timestamp", "date")\
.groupBy("date").sum("response_size").orderBy(col("date")).limit(20)
print("Размер ответов в каждый день:")
response_size_per_date.show()


Топ 10 активных IP адресов:
+---------------+-----+
|             ip|count|
+---------------+-----+
|  55.95.146.189|    1|
| 184.112.10.129|    1|
|  8.206.141.177|    1|
|  84.13.175.225|    1|
|  112.226.233.4|    1|
|179.223.151.182|    1|
|   12.42.129.41|    1|
|  217.138.7.130|    1|
| 86.106.234.105|    1|
| 41.149.217.215|    1|
+---------------+-----+

Количество запросов для каждого метода:
+------+-----+
|method|count|
+------+-----+
|DELETE|25119|
|   GET|25087|
|  POST|24947|
|   PUT|24847|
+------+-----+

Количество запросов с кодом ответа 404: 24935
Размер ответов в каждый день:
+----------+------------------+
|      date|sum(response_size)|
+----------+------------------+
|2024-01-01|           2154079|
|2024-01-02|           2252543|
|2024-01-03|           1923559|
|2024-01-04|           2141237|
|2024-01-05|           2106986|
|2024-01-06|           2152843|
|2024-01-07|           2057161|
|2024-01-08|           2191960|
|2024-01-09|           1878418|
|2024-01-10|  